# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

When finished, you can close the environment.

## 4. Single-Agent Solution

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

## Deep Deterministic Policy Gradients (DDPG)
---


### 1. Import the Necessary Packages

In [1]:
import random as rd
import math
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

#from ddpg_agent_single import Agent


<max_time=1000>

### 2. Instantiate the Environment and Agent

In [2]:
!pip -q install ./python
from unityagents import UnityEnvironment

# select this option to load version 1 (with a single agent) of the environment
#env = UnityEnvironment(file_name='/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64')

# select this option to load version 2 (with 20 agents) of the environment
env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')
    
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
state = env_info.vector_observations.squeeze()
state_size = state.shape[-1]
print('There are {} agents. Each observes a state with length: {}'.format(num_agents, state_size))
print('The state for the first agent looks like:', state[0])

actions = []

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 3.0.36 which is incompatible.
jupyter-console 6.4.3 has requirement jupyter-client>=7.0.0, but you'll have jupyter-client 5.2.4 which is incompatible.


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 20
The state for the first agent looks like: [[  0.00000000e+00  -4.00000000e+00   0.00000000e+00   1.00000000e+00
   -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00  -1.00000000e+01   0.00000000e+00
    1.00000000e+00  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   5.75471878e+00  -1.00000000e+00
    5.55726624e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
   -1.68164849e-01]
 [  0.00000000e+00  -4.00000000e+00   0.00000000e+00   1.00000000e+00
   -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00  -1.00000000e+01   0.00000000

In [10]:
# examine the state space 
state = env_info.vector_observations.squeeze()
state_size = state.shape[-1]
print('There are {} agents. Each observes a state with length: {}'.format(num_agents, state_size))
print('The state for the first agent looks like:\n', np.round(state[0], 3))
print('State shape: ', state.shape)

There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like:
 [  0.     -4.      0.      1.     -0.     -0.     -0.      0.      0.      0.
   0.      0.      0.      0.    -10.      0.      1.     -0.     -0.     -0.
   0.      0.      0.      0.      0.      0.      5.755  -1.      5.557
   0.      1.      0.     -0.168]
State shape:  (20, 33)


In [7]:
state.shape

(20, 33)

In [13]:
print('avg over agents:\n', np.round(np.mean(state, axis=0), 3))
print('std over agents:\n', np.round(np.std(state, axis=0), 3))
print('max over agents:\n', np.round(np.max(state, axis=0), 3))
print('min over agents:\n', np.round(np.min(state, axis=0), 3))

avg over agents:
 [  0.     -4.      0.      1.      0.      0.     -0.      0.      0.      0.
   0.      0.      0.      0.    -10.      0.      1.      0.      0.     -0.
   0.      0.      0.      0.      0.      0.      0.281  -1.      0.281
   0.      1.      0.      0.156]
std over agents:
 [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     5.622  0.     5.678  0.     0.     0.     0.454]
max over agents:
 [  0.     -4.      0.      1.     -0.     -0.     -0.      0.      0.      0.
   0.      0.      0.      0.    -10.      0.      1.     -0.     -0.     -0.
   0.      0.      0.      0.      0.      0.      7.981  -1.      7.981
   0.      1.      0.      0.957]
min over agents:
 [  0.     -4.      0.      1.     -0.     -0.     -0.      0.      0.      0.
   0.      0.      0.      0.    -10.      0.      1.     -0.     -0.     -0.
   0.      0.    

In [16]:
### Actor and Critic models

import torch
import torch.nn as nn
import torch.nn.functional as F

def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)

class Actor(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, seed, fc1_units=400, fc2_units=300):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(Actor, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)
        self.reset_parameters()

    def reset_parameters(self):
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)

    def forward(self, state):
        """Build an actor (policy) network that maps states -> actions."""
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        return torch.tanh(self.fc3(x))

class Critic(nn.Module):
    """Critic (Value) Model."""

    def __init__(self, state_size, action_size, seed, fcs1_units=400, fc2_units=300):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fcs1_units (int): Number of nodes in the first hidden layer
            fc2_units (int): Number of nodes in the second hidden layer
        """
        super(Critic, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fcs1 = nn.Linear(state_size, fcs1_units)
        self.fc2 = nn.Linear(fcs1_units+action_size, fc2_units)
        self.fc3 = nn.Linear(fc2_units, 1)
        self.reset_parameters()

    def reset_parameters(self):
        self.fcs1.weight.data.uniform_(*hidden_init(self.fcs1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)

    def forward(self, state, action):
        """Build a critic (value) network that maps (state, action) pairs -> Q-values."""
        xs = F.relu(self.fcs1(state))
        x = torch.cat((xs, action), dim=1)
        x = F.relu(self.fc2(x))
        return self.fc3(x)
    
class CriticX(nn.Module):
    """Critic (Value) Model."""

    def __init__(self, state_size, action_size, seed, 
                 state_units=192, action_units=64, hidden_units=128):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            state_units (int): Number of nodes in the input layer
            action_units (int): Number of nodes in the input layer
            hidden_units (int): Number of nodes in the second hidden layer
        """
        super(CriticX, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc_state = nn.Linear(state_size, state_units)
        self.fc_action = nn.Linear(action_size, action_units)
        self.fc_hidden = nn.Linear(state_units+action_units, hidden_units)
        self.fc_output = nn.Linear(hidden_units, 1)
        self.reset_parameters()

    def reset_parameters(self):
        self.fc_state.weight.data.uniform_(*hidden_init(self.fc_state))
        self.fc_action.weight.data.uniform_(*hidden_init(self.fc_action))
        self.fc_hidden.weight.data.uniform_(*hidden_init(self.fc_hidden))
        self.fc_output.weight.data.uniform_(-3e-3, 3e-3)

    def forward(self, state, action):
        """Build a critic (value) network that maps (state, action) pairs -> Q-values."""
        xs = F.relu(self.fc_state(state))
        xa = F.relu(self.fc_action(action))
        x = torch.cat((xs, xa), dim=1)
        x = F.relu(self.fc_hidden(x))
        return self.fc_output(x)

In [17]:
### Agent of four networks: remote and local Critics, remote and local Actors

import numpy as np
import random
import copy
from collections import namedtuple, deque

import torch
import torch.nn.functional as F
import torch.optim as optim

BUFFER_SIZE = int(6.4e5)  # replay buffer size = int(1e6)
BATCH_SIZE = 64        # minibatch size = 128
GAMMA = 0.99          # discount factor = 0.99
TAU = 3e-3              # for soft update of target parameters = 1e-3
LR_ACTOR = 1e-4         # learning rate of the actor  = 1e-4
LR_CRITIC = 1e-3        # learning rate of the critic = 3e-4
WEIGHT_DECAY = 1e-5     # L2 weight decay = 0.0001 to 0.
random_seed = 17

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Agent():
    """Interacts with and learns from the environment."""
    
    def __init__(self, state_size, action_size, random_seed):
        """Initialize an Agent object.
        
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
            random_seed (int): random seed
        """
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(random_seed)

        # Actor Network (w/ Target Network)
        self.actor_local = Actor(state_size, action_size, random_seed).to(device)
        self.actor_target = Actor(state_size, action_size, random_seed).to(device)
        self.actor_optimizer = optim.Adam(self.actor_local.parameters(), lr=LR_ACTOR)

        # Critic Network (w/ Target Network)
        self.critic_local = CriticX(state_size, action_size, random_seed).to(device)
        self.critic_target = CriticX(state_size, action_size, random_seed).to(device)
        self.critic_optimizer = optim.Adam(self.critic_local.parameters(), lr=LR_CRITIC, 
                                           weight_decay=WEIGHT_DECAY)

        # Noise process
        self.noise = OUNoise(action_size, random_seed)

        # Replay memory
        self.memory = PriorityReplay(action_size, BUFFER_SIZE, BATCH_SIZE, random_seed)
        #self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, random_seed)
    
    def step(self, step, state, action, reward, next_state, done):
        """Save experience in replay memory, and use random sample from buffer to learn."""
        if len(self.memory) > BATCH_SIZE:
            if reward > 0.:
                ### Save experience ONLY if reward? <<< TO DO 
                self.memory.add(step, state, action, reward, next_state, done)
            
            # Learn from memory, as soon as n_experiences > batch_size
            ##### Learn only part time? TO DO:
            if step % 3 == 0 or done:
                experiences = self.memory.sample(BATCH_SIZE)
                self.learn(experiences, GAMMA)
        else:
            self.memory.add(step, state, action, reward, next_state, done)

    def act(self, state, add_noise=True):
        """Returns actions for given state as per current policy."""
        state = torch.from_numpy(state).float().to(device)
        ### Using LOCAL actor ### <<< TO DO?
        self.actor_local.eval()
        with torch.no_grad():
            action = self.actor_local(state).cpu().data.numpy()
        self.actor_local.train()
        if add_noise:
            action += self.noise.sample()# * 0.75 # fraction bc clipping alot
        return np.clip(action, -1., 1.)

    def reset(self):
        self.noise.reset()

    def learn(self, experiences, gamma):
        """Update policy and value parameters using given batch of experience tuples.
        Q_targets = r + γ * critic_target(next_state, actor_target(next_state))
        where:
            actor_target(state) -> action
            critic_target(state, action) -> Q-value

        Params
        ======
            experiences (Tuple[torch.Tensor]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences

        # ---------------------------- update critic ---------------------------- #
        # Get predicted next-state actions and Q values from target models
        actions_next = self.actor_target(next_states)
        Q_targets_next = self.critic_target(next_states, actions_next)
        # Compute Q targets for current states (y_i)
        ####### TO DO What to do about the dones? 
        ####### Log of all rewards? Pre-scale?
        Q_targets = rewards + gamma * Q_targets_next * (1 - dones)# + dones/100.)
        
        # Compute critic loss
        Q_expected = self.critic_local(states, actions)
        critic_mse_loss = F.mse_loss(Q_expected, Q_targets)
        
        # Minimize critic loss
        self.critic_optimizer.zero_grad()
        critic_mse_loss.backward()
        self.critic_optimizer.step()

        # ---------------------------- update actor ---------------------------- #
        # Compute actor loss
        actions_pred = self.actor_local(states)
        actor_loss = -self.critic_local(states, actions_pred).mean()
        # Minimize actor loss
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        # ------------------- soft update target networks ----------------------- #
        self.soft_update(self.critic_local, self.critic_target, TAU)
        self.soft_update(self.actor_local, self.actor_target, TAU)                     

        #return critic_mse_loss.tolist()

    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target

        Params
        ======
            local_model: PyTorch model (weights will be copied from)
            target_model: PyTorch model (weights will be copied to)
            tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

class OUNoise:
    """Ornstein-Uhlenbeck process."""

    def __init__(self, size, seed, mu=0., theta=0.15, sigma=0.2):
        """Initialize parameters and noise process."""
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.seed = random.seed(seed)
        self.reset()

    def reset(self):
        """Reset the internal state (= noise) to mean (mu)."""
        self.state = copy.copy(self.mu)

    def sample(self):
        """Update internal state and return it as a noise sample."""
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.array([random.random() for _ in range(len(x))])
        self.state = x + dx
        return self.state

class PriorityReplay:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a PriorityReplay object.
        Params
        ======
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
        """
        self.action_size = action_size
        self.batch_size = batch_size
        self.memory = deque(maxlen=buffer_size)  # internal memory (deque)
        self.sum_priority = 1e-6
        self.seed = random.seed(seed)
        self.experience = namedtuple("Experience", field_names=["step",
                                                                "state", 
                                                                "action", 
                                                                "reward", 
                                                                "next_state", 
                                                                "done",
                                                                "priority"])

    def add(self, step, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        priority = (1000 - step + int(done)) * (1 + abs(reward))
        self.sum_priority += priority
        e = self.experience(step, state, action, reward, next_state, done, priority)
        self.memory.append(e)

    def sample(self, n=BATCH_SIZE):
        """Randomly sample a batch of experiences from memory."""
        probs = [m.priority / self.sum_priority for m in self.memory]
        experiences = random.choices(population=self.memory, k=n, weights=probs)
        
        states = torch.from_numpy(np.vstack([e.state for e in experiences])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences]).astype(np.uint8)).float().to(device)
        
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.
        Params
        ======
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  # internal memory (deque)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["step", "state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, step, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(step, state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)
        
        #steps = torch.from_numpy(np.vstack([e.step for e in experiences if e is not None])).float().to(device)
        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)

        return (states, actions, rewards, next_states, dones)
                #steps, 

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

In [18]:
seed = 17
agent = Agent(state_size, action_size, seed)

### 3. Train the Agent with DDPG

In [ ]:
def ddpg(n_episodes=100, print_every=100):
    scores_deque = deque(maxlen=print_every)
    scores = []
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
        state = env_info.vector_observations.squeeze()        # get the current state (for each agent)
        score = 0.                                            # initialize the score (for each agent)
        while True:
            action = agent.act(state)                          # select an action (for each agent)
            #action = np.clip(action, -1, 1)                    # all actions between -1 and 1
            env_info = env.step(action)[brain_name]           # send all actions to tne environment
            next_state = env_info.vector_observations.squeeze() # get next state (for each agent)
            reward = env_info.rewards[0]                        # get reward (for each agent)
            done = env_info.local_done[0]                       # see if episode finished
            agent.step(state, action, reward, next_state, done) # step agent
            score += reward                                   # update the score (for each agent)
            state = next_state                               # roll over states to next time step
            if done:                                        # exit loop if episode finished
                break                                
        scores_deque.append(score)
        scores.append(score)
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)), end="")
        #torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
        #torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
        if i_episode % print_every == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))            
    return scores


In [19]:
def train(n_episodes=2000, print_every=100, max_score=-np.inf):
    scores_deque = deque(maxlen=100)
    scores = []
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
        state = env_info.vector_observations.squeeze()        # get the current state (for each agent)
        score = 0.                                            # initialize the score (for each agent)
        step = 0
        while True:
            step += 1
            action = agent.act(state, add_noise=True)           # select an action (for each agent)
            #action = np.clip(action, -1., 1.)          # Already done : all actions between -1 and 1
            env_info = env.step(action)[brain_name]           # send all actions to tne environment
            next_state = env_info.vector_observations.squeeze() # get next state (for each agent)
            reward = env_info.rewards[0]                        # get reward (for each agent)
            done = env_info.local_done[0]                       # see if episode finished
            agent.step(step, state, action, reward, next_state, done) # step agent
            score += reward                                     # update the score (for each agent)
            state = next_state                                  # roll over states to next time step
            if done:                                            # exit loop if episode finished
                break   
        scores_deque.append(score)
        scores.append(score)
        if score > max_score:
            #torch.save(agent.actor_local.state_dict(), 'highscore_actor.pth')
            #torch.save(agent.critic_local.state_dict(), 'highscore_critic.pth')
            print('\rEpisode {}\tNEW HIGH SCORE! {:.2f}\t   .\t    .        .'.format(i_episode, score))
            max_score = score           
        print('\rEpisode {}\tAverage Score: {:.2f}\tScore: {:.2f}'.format(i_episode, np.mean(scores_deque), score), end="")
        if i_episode % print_every == 0:
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            print('\rEpisode {}\tAverage Score: {:.2f}\tHigh Score: {:.2f}\n'.format(i_episode, np.mean(scores_deque), max_score))   
            
    return scores, max_score

In [ ]:
#scores = ddpg(n_episodes=64, print_every=32)
scores, mx = train(n_episodes=60, print_every=20)
#scores += ores
scores[-8:], mx

In [ ]:
[(e.step, e.reward, e.done) for e in memory if e.reward > 0.0]

In [ ]:
priority = [(1000-e.step) * (e.reward + 0.0001) for e in memory]
prim = [(p,e) for p, e in zip(priority, memory)]
sum_priority = sum(priority)
probs = [p / sum_priority for p in priority]
chosen = random.choices(population=prim, k=BATCH_SIZE, weights=probs)
[(round(p,1), e.step, round(e.reward,3), e.done) for (p, e) in chosen]

In [ ]:
memory = [agent.memory.memory[i] for i in range(len(agent.memory))]
#experiences = random.sample(agent.memory, k=30)
[round(m.reward,3) for m in memory if m.done]# and m.reward>0.]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

### 4. Watch a Smart Agent!

In [ ]:
agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))

env_info = env.reset(train_mode=True)[brain_name]
#state = env.reset()
for t in range(200):
    action = agent.act(state, add_noise=False)
    env.render()
    state, reward, done, _ = env.step(action)
    if done:
        break 

#env.close()

In [ ]:
#### Display from DQN notebook
rewards = []
steps = []
tries = 100
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
img = plt.imshow(env.render())
for j in range(1000):
    action = agent.act(state)
    state, reward, done, trun, info = env.step(action)
    rewards += [reward]
    img.set_data(env.render()) 
    plt.axis('off')
    display.display(plt.gcf())
    display.clear_output(wait=True)
    if done or trun:
        break 

In [ ]:
#env.close()